                                                                                                                        © Moon

# Project Report: Evaluation of the Centrality Algorithm, PageRank

## Introduction

Centrality algorithms are one of the categories of graph algorithms. They identify the important nodes in a given graph and those nodes are defined as vertices with many direct or indirect connections.  One of the centrality algorithms is called $PageRank(PR)$ which identify most important vertices of a graph by measuring the direct influence of nodes based on proportional rank. Now days, developers utilize on analytics, web, social networks, and etc. Google rank websites in their Search engine results.
